Here we show how to define your own residual function and generate it into a factor to use from C++. You can define factors by importing existing symbolic factors that we've defined and modifying them, or by writing your own symbolic functions from scratch using our symbolic toolkit.

In [1]:
import symforce.symbolic as sf
from symforce.codegen import geo_factors_codegen


def custom_between_factor_residual(
    nav_T_src: sf.Pose3,
    nav_T_target: sf.Pose3,
    target_T_src_prior: sf.Pose3,
    prior_weight: sf.Scalar,
    prior_sigmas: sf.Vector6,
    epsilon: sf.Scalar,
) -> sf.Vector6:
    """
    Return the 6dof residual on the relative pose between the given two views. Compares
    the relative pose between the optimized poses to the relative pose between the priors.

    This is similar to geo_factors_codegen.between_factor, but it uses a weight and diagonal
    covariance instead of a sqrt information matrix

    Args:
        nav_T_src: Current pose of the src frame
        nav_T_target: Current pose of the target frame
        target_T_src_prior: Prior on the pose of src in the target frame
        prior_weight: The weight of the Gaussian prior
        prior_sigmas: The diagonal of the sqrt covariance matrix
        epsilon: Small positive value

    Outputs:
        res: The residual of the between factor
    """
    # Note: sqrt(weight) is safe and does not need to be pushed away from 0 by epsilon because
    # weight is a hyperparameter, so we don't need to differentiate with respect to weight or worry
    # about it being slightly negative.  Plus, adding epsilon would break the weight==0 case
    sqrt_info = sf.sqrt(prior_weight) * sf.Matrix66.diag(
        prior_sigmas.applyfunc(lambda s: 1 / (s + epsilon)).to_flat_list()
    )

    return sf.Vector6(
        geo_factors_codegen.between_factor(
            nav_T_target, nav_T_src, target_T_src_prior, sqrt_info, epsilon
        )
    )

In [2]:
from symforce import codegen
from symforce import logger
from symforce import typing as T

def generate(output_dir: T.Openable) -> None:
    """
    Generate the example custom_between_factor_residual factor

    This is called from symforce/test/symforce_examples_custom_factor_generation_codegen_test.py to
    actually generate the factor
    """
    logger.debug("Generating factors")
    namespace = "custom_factor_generation"

    codegen.Codegen.function(
        func=custom_between_factor_residual, config=codegen.CppConfig()
    ).with_linearization(which_args=["nav_T_src", "nav_T_target"]).generate_function(
        output_dir=output_dir, namespace=namespace
    )

generate(".")

codegen.__init__():141 WARNING -- 
    Generating code with epsilon set to 0 - This is dangerous!  You may get NaNs, Infs,
    or numerically unstable results from calling generated functions near singularities.

    In order to safely generate code, you should set epsilon to either a symbol
    (recommended) or a small numerical value like `sf.numeric_epsilon`.  You should do
    this before importing any other code from symforce, e.g. with

        import symforce
        symforce.set_epsilon_to_symbol()

    or

        import symforce
        symforce.set_epsilon_to_number()

    For more information on use of epsilon to prevent singularities, take a look at the
    Epsilon Tutorial: https://symforce.org/tutorials/epsilon_tutorial.html

codegen.__init__():141 WARNING -- 
    Generating code with epsilon set to 0 - This is dangerous!  You may get NaNs, Infs,
    or numerically unstable results from calling generated functions near singularities.

    In order to safely generate code